In [4]:
# 📦 Import
from src.dataset_loader import get_dataloader
from src.train import train_model
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
import torch.optim as optim
import torch
import os
import matplotlib.pyplot as plt
import json

# ⚙️ Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_path = "/content/dataset_prepared/train"
dataset, _ = get_dataloader(data_dir=train_path, batch_size=32)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True
)

# 🧠 Modello pre-addestrato + fine-tuning solo sul layer fc
model = resnet18(weights=ResNet18_Weights.DEFAULT)

# ❄️ Congela tutti i layer
for param in model.parameters():
    param.requires_grad = False

# 🔓 Sblocca solo il classificatore finale
model.fc = nn.Linear(model.fc.in_features, 2)
for param in model.fc.parameters():
    param.requires_grad = True

# Sposta il modello sulla GPU/CPU
model = model.to(device)

# 🔧 Ottimizzatore solo per fc
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)

# 🔁 Training
history, run_dir = train_model(model, dataloader, criterion, optimizer, device, epochs=10, resume=False)
#set resume to true if you want to load a checkpoint

# 📝 Salva note/commenti sulla run
with open(os.path.join(run_dir, "run_notes.txt"), "w") as f:
    f.write("🧪 Fine-tuning binario con ResNet18 (solo fc) - 10 epoche, lr=1e-4\n")
    f.write("📌 Dataset: IDC binario - 50x50 patch\n")
    f.write("✅ Notebook: 02_training.ipynb\n")

# 📊 Mostra F1-score
plt.plot(history["f1"])
plt.title("📈 F1-score per epoca")
plt.xlabel("Epoch")
plt.grid()
plt.show()

# 📂 Visualizza path della cartella risultati
print(f"📁 I risultati sono salvati in: {run_dir}")

# (opzionale) Visualizza contenuto del file train_history.json
with open(os.path.join(run_dir, "train_history.json")) as f:
    metrics = json.load(f)

print("📊 F1 finale:", metrics["f1"][-1])

import shutil

# Zippa la cartella
run_zip_path = f"{run_dir}.zip"
shutil.make_archive(run_dir, 'zip', run_dir)

try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    files.download(run_zip_path)
else:
    print("⛔ Download disabilitato: non sei su Colab.")



SyntaxError: incomplete input (1288183229.py, line 80)